In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.ensemble import StackingRegressor

from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [3]:
X = housing['data']
y = housing['target']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y , train_size = 0.7)

In [5]:
individual_models = [SVR() , KNeighborsRegressor(), DecisionTreeRegressor()  ]

In [6]:
for model in individual_models:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print("{} : RMSE : {:.3f}, Error Variance : {:.3f}".format(type(model).__name__ , np.sqrt(mean_squared_error(y_pred, y_test)), np.var(np.abs(y_test-y_pred))))

SVR : RMSE : 1.1565173557373802, Error Variance : 0.5921918247784349
KNeighborsRegressor : RMSE : 1.066398256423146, Error Variance : 0.4610583137881506
DecisionTreeRegressor : RMSE : 0.7184783745604179, Error Variance : 0.3119167342568308


In [7]:
base_models = [(type(i).__name__, i)for i in individual_models]
meta_model = LinearRegression()

In [8]:
stacked_model = StackingRegressor(estimators = base_models , final_estimator = meta_model,
                                  cv = 3, n_jobs = -1, passthrough = False)

In [9]:
stacked_model.fit(X_train, y_train)

StackingRegressor(cv=3,
                  estimators=[('SVR', SVR()),
                              ('KNeighborsRegressor', KNeighborsRegressor()),
                              ('DecisionTreeRegressor',
                               DecisionTreeRegressor())],
                  final_estimator=LinearRegression(), n_jobs=-1)

In [10]:
y_pred = stacked_model.predict(X_test)

In [11]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.6680076310235183

In [12]:
np.var(np.abs(y_test-y_pred))

0.23435315946843152